In [1]:
import os
from framework import PandasBB as pbb

In [2]:
if os.environ['AMBIENTE'] == 'MODELAGEM':
    print('INFO: Carregando as credenciais de desenvolvimento...')
    uid  = 'DB2I127A'
    pwd  = '99689720'
    host = 'gwdb2.bb.com.br'
    port = '50100'
    db   = 'BDB2P04'
else:
    print('INFO: Carregando as credenciais de produção...')
    try:
        os.environ['DB2_PORT']
    except:
        os.environ['DB2_PORT'] = os.environ['DB2_PORTA']
    uid  = os.environ['DB2_USER']
    pwd  = os.environ['DB2_PASSWORD']
    host = os.environ['DB2_HOST']
    port = os.environ['DB2_PORT']
    db   = os.environ['DB2_DATABASE']

INFO: Carregando as credenciais de desenvolvimento...


In [3]:
sql = """
SELECT 
    DT_SDO AS DT_REF, 
    NM_TIP_PBCO,
    sum(VL_SDO) AS VL_SDO
FROM DB2T5A.SDO_ENG_RNV_MSL 
WHERE DT_SDO >= (SELECT ADD_MONTHS(MAX(DT_SDO), -23) DT_SDO FROM DB2T5A.SDO_ENG_RNV_MSL)
GROUP BY DT_SDO, NM_TIP_PBCO
ORDER BY DT_SDO DESC
"""

df_pre = pbb.select_db2(uid, pwd, sql)

/projeto/bc49e8e1-5396-4bfd-9e28-6a9dc2aaaf89/workdir/eng_cops/cops.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pandas.read_sql(query, dbi)


In [4]:
tbl = "SELECT * FROM DB2I1014.CNSD_SDO_ENG_RNV"
user = "DB2I1014"
password = "87025801"

df_tbl = pbb.select_db2(user, password, tbl)

/projeto/bc49e8e1-5396-4bfd-9e28-6a9dc2aaaf89/workdir/eng_cops/cops.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pandas.read_sql(query, dbi)


In [5]:
# Conta a quantidade de registros na origem
qtd = len(df_pre.index)

In [6]:
# Valida se existe a data atual de carga na tabela final
dt = pbb.max_date_exists(df_pre, df_tbl, 'DT_REF')

In [7]:
# Valida se a quantidade de segmentos na carga está correta
df_g = df_pre
g = pbb.count_by_group(df_g[df_g['DT_REF']==df_pre['DT_REF'].max()], 'NM_TIP_PBCO')

In [8]:
# Agrupa os dados pelo saldo mensal
df_group = df_pre.groupby('DT_REF', as_index=False)['VL_SDO'].sum()
df_sdo = df_group.sort_values(by='DT_REF', ascending=False)

In [9]:
# Calcula o percentual variação
df_pct = df_group
df_out = pbb.percent_variation(df_pct, 'VL_SDO', 10)
qtd_pct = len(df_out.index)

In [10]:
load = False
sts = 0
if qtd > 0:
    if dt == False:
        if qtd_pct == 0:            
            if g == 4:
                load = True
            else:
                sts = 3
                load = False
        else:
            sts = 4
            load = False
    else:
        sts = 2
        load = False
else:
    sts = 5
    load = False

In [13]:
df_sdo['CD_EST'] = sts

In [17]:
# Cria colunas da carga
df_ts = pbb.get_timestamp(df_sdo)

In [19]:
# Renomeia coluna de timestamp
df_pos = pbb.rename_column(df_ts, 'timestamp', 'TS_ATL')

In [23]:
# Ordena colunas para gravação
df = pbb.order_columns(df_pos, df_tbl)

In [ ]:
table = 'DB2I1014.CNSD_SDO_ENG_RNV'

In [22]:
pbb.insert_db2(user, password, df, table)

Conexão criada.
INSERT INTO DB2I1014.CNSD_SDO_ENG_RNV (TS_ATL, DT_REF, VL_SDO, CD_EST) VALUES (?, ?, ?, ?)
Dados inseridos com sucesso a partir do DataFrame.
Conexão fechada.


In [ ]:
# end